In [1]:
# Import Libraries
import numpy as np

from project_utils.bb_model         import BB_Model
from project_utils.feature_stats    import Feature_Statistics
from project_utils.feature_stats    import Regression_Container
from project_utils.acq_data_capture import Acq_Data
from project_utils.surroagte_gp     import Surrogate_GP

Num_Points = 21


In [2]:
# Load Boston model

Boston_Data = BB_Model('Boston')

#Boston_Data.df().head()

In [3]:
X_train, X_test, y_train, y_test = Boston_Data.get_TT_data()



In [4]:
Boston_Data.MPL()

-1.3649339947290051


MLPRegressor(random_state=RandomState(MT19937) at 0x18B083F9840)

In [5]:
Boston_Data.Random_Forest()

0.8549233671381437


RandomForestRegressor(random_state=RandomState(MT19937) at 0x18B083F9840)

In [6]:
Boston_Data.GP()

-7.127467928794353


GaussianProcessRegressor()

In [7]:
print(Boston_Data.get_features())

['crime_rate', 'zoned_lots', 'industry', 'by_river', 'NOX', 'avg_rooms', 'pre_1940', 'emp_distance', 'rad_access', 'tax_rate', 'pupil_tea_rat', 'low_status']


In [8]:
#Default settings
from unravel.tabular import UnRAVELTabularExplainer

X_train, X_test, y_train, y_test = Boston_Data.get_TT_data()


#Initializing the explainer object
Unravel_Exp = UnRAVELTabularExplainer(bbox_model           = Boston_Data.get_Random_Forest(),
                                      train_data           = X_train,
                                      categorical_features = Boston_Data.get_catagorical(),
                                      mode                 = Boston_Data.get_mode())       

sur_GP = Surrogate_GP()


warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [9]:
# Generate explantion data for Random Forest

Boston_Stats = Feature_Statistics(feature_names = Boston_Data.get_features(), mode = Boston_Data.get_mode())

if Num_Points == 0: Num_Points = X_test.shape[0]

for test_instance in range(Num_Points):

    print('INSTANCE: ', test_instance)
       
    data_row  = np.array([X_test[test_instance]])
    
    scores_exp = Unravel_Exp.explain(X_init            = data_row,
                                     feature_names     = Boston_Data.get_features(),
                                     alpha             = "FUR",
                                     max_iter          = 20,
                                     normalize         = False,
                                     importance_method = "KL")
    
    #print(scores_exp)
       
    f_predict = Boston_Data.get_Random_Forest().predict(data_row.reshape(1, -1))

    #e_predict = Unravel_Exp.gpmodel_predict(data_row)
    
    acq_data = Unravel_Exp.get_acq_data()
    
    X, y = Unravel_Exp.get_surrogate_data()

    e_predict = sur_GP.predict(X, y)
    
    e_predict = np.array(e_predict).ravel()
    
    print(f_predict, e_predict)
    

    Boston_Stats.Add_Sample(sample_scores = scores_exp,
                            outcome       = y_test[test_instance],
                            X_row         = data_row,
                            f_prediction  = f_predict,
                            e_prediction  = e_predict,
                            model         = None)
    

INSTANCE:  0


<class 'ValueError'>: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 12

In [ ]:
stop!!

In [ ]:
Boston_Stats.Feature_Counts(max_features=8, scaled=True, threshold=0.075)

In [ ]:
Boston_Stats.Frequency_Plot()

In [ ]:
Boston_Stats.Box_Plot()

In [ ]:
Boston_Stats.Violin_Plot(top_features=False)

In [ ]:
Boston_Container = Regression_Container(Boston_Stats, 5)

Boston_Container.Feature_Counts(max_features=11, scaled=True, threshold=0.05)


In [ ]:
Boston_Container.Frequency_Plot()

In [ ]:
Boston_Stats.Fidelity()

In [ ]:
Boston_Stats.Jaccard_Values()

In [ ]:
Boston_Stats.Reg_Fidelity_Graph()